In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from delta.tables import DeltaTable

In [0]:
bronze_orders_tbl = "my_catalog.bronze_retailx.orders"
silver_orders_tbl = "my_catalog.silver_retailx.orders"

silver_customers_tbl = "my_catalog.silver_retailx.customers"

In [0]:
last_processed_ts = (
    spark.table(silver_orders_tbl)
    .select(max("_ingested_at").alias("max_ts"))
    .collect()[0]["max_ts"]
)

if last_processed_ts is None:
    last_processed_ts = "1900-01-01"

In [0]:
bronze_df = (
    spark.table(bronze_orders_tbl)
    .filter(col("_ingested_at") > lit(last_processed_ts))
)

In [0]:
display(bronze_df)

ORDER_ID,CUSTOMER_ID,ORDER_DATE,AMOUNT,_rescued_data,_ingested_at,_source_file
116,6,2025-04-14,1250,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
117,6,2025-04-29,980,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
118,6,2025-05-18,2100,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
119,7,2025-04-16,1450,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
120,7,2025-05-01,1900,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
121,7,2025-05-20,2600,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
122,8,2025-04-19,1750,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
123,8,2025-05-04,2150,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
124,8,2025-05-22,3000,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv
125,9,2025-04-21,1350,null,2025-12-14T06:00:01.202Z,/Volumes/my_catalog/raw_retailx/orders/2025/12/13/orders%202.csv


In [0]:
df = bronze_df.drop("_rescued_data", "_source_file")
display(df)

ORDER_ID,CUSTOMER_ID,ORDER_DATE,AMOUNT,_ingested_at
116,6,2025-04-14,1250,2025-12-14T06:00:01.202Z
117,6,2025-04-29,980,2025-12-14T06:00:01.202Z
118,6,2025-05-18,2100,2025-12-14T06:00:01.202Z
119,7,2025-04-16,1450,2025-12-14T06:00:01.202Z
120,7,2025-05-01,1900,2025-12-14T06:00:01.202Z
121,7,2025-05-20,2600,2025-12-14T06:00:01.202Z
122,8,2025-04-19,1750,2025-12-14T06:00:01.202Z
123,8,2025-05-04,2150,2025-12-14T06:00:01.202Z
124,8,2025-05-22,3000,2025-12-14T06:00:01.202Z
125,9,2025-04-21,1350,2025-12-14T06:00:01.202Z


In [0]:
df = (
    df
    # data type fixes
    .withColumn("order_id", col("ORDER_ID").cast("bigint"))
    .withColumn("customer_id", col("CUSTOMER_ID").cast("bigint"))
    .withColumn("order_date", to_date("ORDER_DATE"))
    .withColumn("amount", col("AMOUNT").cast("decimal(10,2)"))
    .withColumn("_ingested_at", to_timestamp(col("_ingested_at")))
)
display(df)

order_id,customer_id,order_date,amount,_ingested_at
116,6,2025-04-14,1250.00,2025-12-14T06:00:01.202Z
117,6,2025-04-29,980.00,2025-12-14T06:00:01.202Z
118,6,2025-05-18,2100.00,2025-12-14T06:00:01.202Z
119,7,2025-04-16,1450.00,2025-12-14T06:00:01.202Z
120,7,2025-05-01,1900.00,2025-12-14T06:00:01.202Z
121,7,2025-05-20,2600.00,2025-12-14T06:00:01.202Z
122,8,2025-04-19,1750.00,2025-12-14T06:00:01.202Z
123,8,2025-05-04,2150.00,2025-12-14T06:00:01.202Z
124,8,2025-05-22,3000.00,2025-12-14T06:00:01.202Z
125,9,2025-04-21,1350.00,2025-12-14T06:00:01.202Z


In [0]:
clean_orders = (
    df
    .filter(col("order_id").isNotNull())
    .filter(col("customer_id").isNotNull())
    .filter(col("amount") >= 0)
)

In [0]:
df = (
    df
    .withColumn(
        "rn",
        row_number().over(
            Window.partitionBy("order_id")
            .orderBy(col("order_id").asc())
        )
    )
    .filter("rn = 1")
    .drop("rn")
)

In [0]:
display(df)

order_id,customer_id,order_date,amount,_ingested_at
116,6,2025-04-14,1250.00,2025-12-14T06:00:01.202Z
117,6,2025-04-29,980.00,2025-12-14T06:00:01.202Z
118,6,2025-05-18,2100.00,2025-12-14T06:00:01.202Z
119,7,2025-04-16,1450.00,2025-12-14T06:00:01.202Z
120,7,2025-05-01,1900.00,2025-12-14T06:00:01.202Z
121,7,2025-05-20,2600.00,2025-12-14T06:00:01.202Z
122,8,2025-04-19,1750.00,2025-12-14T06:00:01.202Z
123,8,2025-05-04,2150.00,2025-12-14T06:00:01.202Z
124,8,2025-05-22,3000.00,2025-12-14T06:00:01.202Z
125,9,2025-04-21,1350.00,2025-12-14T06:00:01.202Z


In [0]:
valid_customers = (
    spark.table(silver_customers_tbl)
    .filter("is_current = true")
    .select("customer_id")
)

clean_orders = clean_orders.join(
    valid_customers,
    "customer_id",
    "inner"   # rejects orphan orders
)

**Assumption:** Orders are immutable, Once an order is created, it is not updated. 


In [0]:
clean_orders.select(
    "order_id",
    "customer_id",
    "order_date",
    "amount",
    "_ingested_at"
).write.format("delta") \
 .mode("append") \
 .saveAsTable(silver_orders_tbl)

In [0]:
df = spark.table(silver_orders_tbl)
display(df)

order_id,customer_id,order_date,amount,_ingested_at
116,6,2025-04-14,1250.00,2025-12-14T06:00:01.202Z
117,6,2025-04-29,980.00,2025-12-14T06:00:01.202Z
118,6,2025-05-18,2100.00,2025-12-14T06:00:01.202Z
119,7,2025-04-16,1450.00,2025-12-14T06:00:01.202Z
120,7,2025-05-01,1900.00,2025-12-14T06:00:01.202Z
121,7,2025-05-20,2600.00,2025-12-14T06:00:01.202Z
122,8,2025-04-19,1750.00,2025-12-14T06:00:01.202Z
123,8,2025-05-04,2150.00,2025-12-14T06:00:01.202Z
124,8,2025-05-22,3000.00,2025-12-14T06:00:01.202Z
125,9,2025-04-21,1350.00,2025-12-14T06:00:01.202Z


In [0]:
# Recommended for large volumes
# PARTITIONED BY (order_year, order_month)